# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using PseudoPotentialData

pseudopotentials = PseudoFamily("dojo.nc.sr.pbesol.v0_4_1.standard.upf")
model = model_DFT(bulk(:Si); functionals=PBEsol(), pseudopotentials)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -8.397884887044                   -0.90    5.5   96.1ms
  2   -8.400242587560       -2.63       -1.74    1.0   19.7ms
  3   -8.400405430400       -3.79       -2.98    1.5   20.4ms
  4   -8.400427865038       -4.65       -2.98    3.2   25.5ms
  5   -8.400427979300       -6.94       -3.08    1.0   19.6ms
  6   -8.400428146628       -6.78       -4.73    1.0   19.5ms
  7   -8.400428151860       -8.28       -4.49    2.5   23.7ms
  8   -8.400428152195       -9.47       -5.45    1.0   48.6ms
  9   -8.400428152208      -10.90       -5.91    1.2   20.8ms
 10   -8.400428152209      -11.94       -6.48    1.5   21.5ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime 
---   ---------------   ---------   ---------   ----   ----   ------
  1   -8.397836603197                   -0.90           5.0    1.78s
  2   -8.400387580449       -2.59       -1.79   0.80    2.2    593ms
  3   -8.400423376094       -4.45       -3.01   0.80    1.0    220ms
  4   -8.400428111935       -5.32       -3.45   0.80    2.5   20.9ms
  5   -8.400428149814       -7.42       -4.58   0.80    1.2   17.8ms
  6   -8.400428152177       -8.63       -5.52   0.80    2.0   19.8ms
  7   -8.400428152208      -10.51       -6.79   0.80    2.8   21.4ms


## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

┌ Warning: x_tol is deprecated. Use x_abstol or x_reltol instead. The provided value (-1) will be used as x_abstol.
└ @ Optim ~/.julia/packages/Optim/gmigl/src/types.jl:110
┌ Warning: f_tol is deprecated. Use f_abstol or f_reltol instead. The provided value (-1) will be used as f_reltol.
└ @ Optim ~/.julia/packages/Optim/gmigl/src/types.jl:120
n     Energy            log10(ΔE)   log10(Δρ)   Δtime 
---   ---------------   ---------   ---------   ------
  1   +1.088566279408                   -1.12    3.50s
  2   -1.782288768995        0.46       -0.69    152ms
  3   -4.386824222774        0.42       -0.44   44.7ms
  4   -5.972195779149        0.20       -0.50   44.3ms
  5   -7.453864487895        0.17       -0.76   44.0ms
  6   -7.859067645401       -0.39       -1.25   63.2ms
  7   -8.165722450232       -0.51       -1.45   33.0ms
  8   -8.255628480555       -1.05       -1.80   33.1ms
  9   -8.315582342720       -1.22       -2.13   32.9ms
 10   -8.343798837917       -1.55       -2.00   3

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime 
---   ---------------   ---------   ---------   ----   ------
  1   -8.397918446042                   -0.90    5.2   27.4ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime 
---   ---------------   ---------   ---------   ------
  1   -8.400427990921                   -1.79    8.02s
  2   -8.400428152209       -6.79       -4.04    3.13s
  3   -8.400428152209      -14.27       -7.86   73.4ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 1.7253381156527988e-7
|ρ_newton - ρ_scfv| = 3.637279477857947e-7
|ρ_newton - ρ_dm|   = 3.2187445198222543e-6
